### Import Packages

In [8]:
library(tidyr)

### Import Data

In [50]:
data=read.delim("Integrated_Data_Formatted.txt", header = TRUE, sep = "\t", dec = ".")
data_f <- data[,-1]
rownames(data_f) <- data[,1]

### Functions

In [1]:
# extract p-value from model
overall_p <- function(my_model) {
    f <- summary(my_model)$fstatistic
    p <- pf(f[1],f[2],f[3],lower.tail=F)
    attributes(p) <- NULL
    return(p)
}

### Univariate Regression Model

In [17]:
# test univariate
data_f_covar=data_f[!is.na(data_f$income_fy23),]
covar=c(data_f_covar$income_fy23)
depVar=c(data_f_covar$Density_Diff_2022_2016)

In [20]:
# test univariate
p_val_list=list()
model <- lm(depVar ~ covar)
summary(model)
p_val=overall_p(model)
p_val_list=append(p_val_list,p_val)


Call:
lm(formula = depVar ~ covar)

Residuals:
    Min      1Q  Median      3Q     Max 
-38.598   0.162   0.468   0.888   2.872 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept)  0.03565    0.80162   0.044    0.965
covar       -0.18641    0.27435  -0.679    0.498

Residual standard error: 3.885 on 184 degrees of freedom
Multiple R-squared:  0.002503,	Adjusted R-squared:  -0.002918 
F-statistic: 0.4617 on 1 and 184 DF,  p-value: 0.4977


In [35]:
# for(i in 1:ncol(data_f)) {       # for-loop over columns
#     data_f_covar=data_f[!is.na(data_f[ , i]),]
#     covar = c(data_f_covar[ , i])
#     print(covar)
# }

In [21]:
colFactors=colnames(data_f)

In [70]:
# run univariate model
p_val_list=list()
storage <- list()
for (i in colnames(data_f)) {
# for(i in 1:ncol(data_f)) {
    data_f_covar=data_f[!is.na(data_f$i),]
    covar=c(data_f_covar$i)
#     data_f_covar=data_f[!is.na(data_f[ , i]),]
#     colVar=colnames(data_f_covar)
#     print(colVar)
    covar = c(data_f_covar[ , i])
    depVar=c(data_f_covar$Density_Diff_2022_2016)
    model <- lm(depVar ~ covar)
#     storage[[i]] <- lm(Density_Diff_2022_2016 ~ get(i), data_f_covar)
    storage[[i]] <- summary(model)
#     summary(model)
    p_val=overall_p(model)
    p_val_list=append(p_val_list,p_val)
}

Warning message in summary.lm(model):
“essentially perfect fit: summary may be unreliable”
Warning message in summary.lm(my_model):
“essentially perfect fit: summary may be unreliable”


In [77]:
# save storage
sink("Univariate_LM_Summaries.txt")
print(storage)
sink()  # returns output to the console

$income_fy23

Call:
lm(formula = depVar ~ covar)

Residuals:
    Min      1Q  Median      3Q     Max 
-38.598   0.162   0.468   0.888   2.872 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept)  0.03565    0.80162   0.044    0.965
covar       -0.18641    0.27435  -0.679    0.498

Residual standard error: 3.885 on 184 degrees of freedom
Multiple R-squared:  0.002503,	Adjusted R-squared:  -0.002918 
F-statistic: 0.4617 on 1 and 184 DF,  p-value: 0.4977


$populationquartile_2021

Call:
lm(formula = depVar ~ covar)

Residuals:
    Min      1Q  Median      3Q     Max 
-37.434  -0.445   0.318   1.095   4.036 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -2.7883     0.6841  -4.076 6.81e-05 ***
covar         0.9141     0.2473   3.696 0.000289 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.753 on 184 degrees of freedom
Multiple R-squared:  0.06912,	Adjusted R-squared:  0.06406 
F-st

In [52]:
# create dataframe of pvalues
uni_p_val_df <- data.frame(unlist(colnames(data_f)), unlist(p_val_list))

In [53]:
# get cols with p values less than 0.10
uni_p_val_df_filt <- uni_p_val_df[uni_p_val_df$unlist.p_val_list.<=0.10,]
uni_p_val_df_filt=uni_p_val_df_filt[-10,]

In [54]:
# save p value dataframes
write.csv(uni_p_val_df, "Univariate_LM_P_Values.csv", row.names=FALSE, quote=FALSE) 
write.csv(uni_p_val_df_filt, "Univariate_LM_P_Values_Filtered.csv", row.names=FALSE, quote=FALSE) 

### Multivariate Regression Model

In [62]:
# import data
# data_tf=read.delim("Integrated_Data_Formatted_TopFactors.txt", header = TRUE, sep = "\t", dec = ".")
# data_tf_f <- data_tf[,-1]
# rownames(data_tf_f) <- data_tf[,1]

# import scaled data
data_tf=read.delim("Integrated_Data_Formatted_TopFactors_Scaled.txt", header = TRUE, sep = "\t", dec = ".")
data_tf_f <- data_tf[,-1]
rownames(data_tf_f) <- data_tf[,1]

In [63]:
# create vectors
populationquartile_2021_vector=c(data_tf_f$populationquartile_2021)
age0to14_percent_vector=c(data_tf_f$age0to14_percent)
age65over_percent_vector=c(data_tf_f$age65over_percent)
deathrate_per1000_vector=c(data_tf_f$deathrate_per1000)
physicians_per1000_vector=c(data_tf_f$physicians_per1000)
risk_impoverishing_surgicalcare_percent_vector=c(data_tf_f$risk_impoverishing_surgicalcare_percent)
healthexpenditure_percentofgdp_vector=c(data_tf_f$healthexpenditure_percentofgdp)
outofpocketexpenditure_percentcurrenthealthexpenditure_vector=c(data_tf_f$outofpocketexpenditure_percentcurrenthealthexpenditure)
outofpocketexpenditure_over25percenthouseholdincome_percentofpeople_vector=c(data_tf_f$outofpocketexpenditure_over25percenthouseholdincome_percentofpeople)
Density_Diff_2022_2016_vector=c(data_tf_f$Density_Diff_2022_2016)

In [65]:
# mv model
mvModel <- lm(Density_Diff_2022_2016_vector ~ populationquartile_2021_vector + age0to14_percent_vector + age65over_percent_vector + deathrate_per1000_vector + physicians_per1000_vector + risk_impoverishing_surgicalcare_percent_vector + healthexpenditure_percentofgdp_vector + outofpocketexpenditure_percentcurrenthealthexpenditure_vector + outofpocketexpenditure_over25percenthouseholdincome_percentofpeople_vector)
summary(mvModel)


Call:
lm(formula = Density_Diff_2022_2016_vector ~ populationquartile_2021_vector + 
    age0to14_percent_vector + age65over_percent_vector + deathrate_per1000_vector + 
    physicians_per1000_vector + risk_impoverishing_surgicalcare_percent_vector + 
    healthexpenditure_percentofgdp_vector + outofpocketexpenditure_percentcurrenthealthexpenditure_vector + 
    outofpocketexpenditure_over25percenthouseholdincome_percentofpeople_vector)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.56066 -0.05511  0.00186  0.04328  0.42323 

Coefficients:
                                                                            Estimate
(Intercept)                                                                 0.463325
populationquartile_2021_vector                                              0.078030
age0to14_percent_vector                                                    -0.069113
age65over_percent_vector                                                    0.062975
deathrate_per1

In [72]:
# save mv model summary
mvModel_Summary=summary(mvModel)